# 유성우 예측

### tl;dr
- 혜성이 지나간 자리에 유성체 stream이 발생하고 이 것이 지구의 공전 궤도와 만나면 유성우가 발생
- 유성우는 관측자의 위치, 관측 시기, 관측 날짜의 달의 위상 등에 따라 관측 여부가 정해짐
- 그럼 내가 있는 도시에서 언제 유성우를 볼 수 있는 지 예측할 수 있을까?

### 유성우 예측을 위해 필요한 정보
- https://github.com/drguthals/learnwithdrg/tree/main/OverTheMoon/meteor-showers/data
- 유성우가 발생하는 기간 (4개의 혜성으로부터 발생하는 주요 5대 유성우): meteorshowers.csv
- 유성우를 관측할 수 있는 위도: constellations.csv
- 달의 위상: moonphases.csv
- 도시 위도: cities.csv

In [39]:
import numpy as np 
import pandas as pd 

In [40]:
meteor_showers = pd.read_csv('data/meteorshowers.csv') 
moon_phases = pd.read_csv('data/moonphases.csv')
constellations = pd.read_csv('data/constellations.csv')
cities = pd.read_csv('data/cities.csv')

In [42]:
change_meteor_shower = {'name':'Chang\'e','radiant':'Draco','bestmonth':'october','startmonth':'october','startday':1,'endmonth':'october','endday':31,'hemisphere':'northern','preferredhemisphere':'northern'}
meteor_showers = pd.concat([meteor_showers, pd.DataFrame([change_meteor_shower])], ignore_index=True)
draco_constellation = {'constellation':'Draco','bestmonth':'july','latitudestart':90,'latitudeend':-15,'besttime':2100,'hemisphere':'northern'}
constellations = pd.concat([constellations, pd.DataFrame([draco_constellation])], ignore_index=True)

In [43]:
meteor_showers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 7 non-null      object
 1   radiant              7 non-null      object
 2   bestmonth            7 non-null      object
 3   startmonth           7 non-null      object
 4   startday             7 non-null      int64 
 5   endmonth             7 non-null      object
 6   endday               7 non-null      int64 
 7   hemisphere           7 non-null      object
 8   preferredhemisphere  7 non-null      object
dtypes: int64(2), object(7)
memory usage: 636.0+ bytes


In [44]:
moon_phases.head()

,month,day,moonphase,specialevent
0,january,1,NaN,NaN
1,january,2,first quarter,NaN
2,january,3,NaN,NaN
3,january,4,NaN,NaN
4,january,5,NaN,NaN


In [45]:
moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   month         366 non-null    object
 1   day           366 non-null    int64 
 2   moonphase     50 non-null     object
 3   specialevent  10 non-null     object
dtypes: int64(1), object(3)
memory usage: 11.6+ KB


In [46]:
constellations.head()

,constellation,bestmonth,latitudestart,latitudeend,besttime,hemisphere
0,Lyra,august,90,-40,21:00,northern
1,Aquarius,october,65,-90,21:00,southern
2,Orion,january,85,-75,21:00,northern
3,Perseus,december,90,-35,21:00,northern
4,Leo,april,90,65,21:00,northern


In [49]:
constellations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   constellation  7 non-null      object
 1   bestmonth      7 non-null      object
 2   latitudestart  7 non-null      int64 
 3   latitudeend    7 non-null      int64 
 4   besttime       7 non-null      object
 5   hemisphere     7 non-null      object
dtypes: int64(2), object(4)
memory usage: 468.0+ bytes


In [48]:
cities.head()

,city,latitude,country
0,Abu Dhabi,24.47,United Arab Emirates
1,Abuja,9.07,Nigeria
2,Accra,5.55,Ghana
3,Adamstown,-25.07,Pitcairn Islands
4,Addis Ababa,9.02,Ethiopia


In [47]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      256 non-null    object 
 1   latitude  256 non-null    float64
 2   country   256 non-null    object 
dtypes: float64(1), object(2)
memory usage: 6.1+ KB


### Data 준비
#### 월을 숫자로 변경

In [55]:
months = {'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}
meteor_showers.bestmonth = meteor_showers.bestmonth.map(months)
meteor_showers.startmonth = meteor_showers.startmonth.map(months)
meteor_showers.endmonth = meteor_showers.endmonth.map(months)
moon_phases.month = moon_phases.month.map(months)
constellations.bestmonth = constellations.bestmonth.map(months)

In [51]:
meteor_showers.head()

,name,radiant,bestmonth,startmonth,startday,endmonth,endday,hemisphere,preferredhemisphere
0,Lyrids,Lyra,april,april,21,april,22,northern,northern
1,Eta Aquarids,Aquarius,may,april,19,may,28,"northern, southern",southern
2,Orionids,Orion,october,october,2,november,7,"northern, southern","northern, southern"
3,Perseids,Perseus,august,july,14,august,24,northern,northern
4,Leonids,Leo,november,november,6,november,30,"northern, southern","northern, southern"


In [52]:
meteor_showers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 7 non-null      object
 1   radiant              7 non-null      object
 2   bestmonth            7 non-null      object
 3   startmonth           7 non-null      object
 4   startday             7 non-null      int64 
 5   endmonth             7 non-null      object
 6   endday               7 non-null      int64 
 7   hemisphere           7 non-null      object
 8   preferredhemisphere  7 non-null      object
dtypes: int64(2), object(7)
memory usage: 636.0+ bytes


#### 월과 일을 묶어 날짜로 변환

In [56]:
meteor_showers['startdate'] = pd.to_datetime(2020*10000+meteor_showers.startmonth*100+meteor_showers.startday,format='%Y%m%d')
meteor_showers['enddate'] = pd.to_datetime(2020*10000+meteor_showers.endmonth*100+meteor_showers.endday,format='%Y%m%d')

In [57]:
moon_phases['date'] = pd.to_datetime(2020*10000+moon_phases.month*100+moon_phases.day,format='%Y%m%d')

#### 기타 데이터 변환

In [58]:
hemispheres = {'northern':0, 'southern':1, 'northern, southern':3}
meteor_showers.hemisphere = meteor_showers.hemisphere.map(hemispheres)
constellations.hemisphere = constellations.hemisphere.map(hemispheres)

In [59]:
phases = {'new moon':0,'third quarter':0.5, 'first quarter':0.5,'full moon':1.0}
moon_phases['percentage'] = moon_phases.moonphase.map(phases)
moon_phases.head()

,month,day,moonphase,specialevent,date,percentage
0,1,1,NaN,NaN,2020-01-01,NaN
1,1,2,first quarter,NaN,2020-01-02,0.5
2,1,3,NaN,NaN,2020-01-03,NaN
3,1,4,NaN,NaN,2020-01-04,NaN
4,1,5,NaN,NaN,2020-01-05,NaN


#### 불필요한 데이터 삭제
다른 Column으로 변경되어 불필요해졌거나 모두 같은 값을 가지고 있는 column (constellations의 besttime)을 삭제한다.

In [60]:
meteor_showers = meteor_showers.drop(['startmonth', 'startday', 'endmonth', 'endday', 'hemisphere'], axis=1)
moon_phases = moon_phases.drop(['month','day','moonphase','specialevent'], axis=1)
constellations = constellations.drop(['besttime'], axis=1)

#### 달의 위상 정보를 채운다.
값이 비어있는 곳의 앞과 뒤의 값을 기준으로 linspace()를 통해 값을 채운다.

In [69]:
# 달의 위상 정보를 사인 함수로 보간한다.
# NaN이 아닌 인덱스 찾기
non_null_mask = moon_phases['percentage'].notna()
non_null_indices = non_null_mask[non_null_mask].index.tolist()

# 사인 함수 기반 보간
for i in range(len(non_null_indices) - 1):
    start_idx = non_null_indices[i]
    end_idx = non_null_indices[i + 1]
    
    start_val = moon_phases.loc[start_idx, 'percentage']
    end_val = moon_phases.loc[end_idx, 'percentage']
    
    # 사이의 인덱스 개수
    num_points = end_idx - start_idx + 1
    
    # 사인 함수로 보간 (0~π 범위)
    x = np.linspace(0, np.pi, num_points)
    interpolated = start_val + (end_val - start_val) * (1 - np.cos(x)) / 2
    
    # 값 채우기
    moon_phases.loc[start_idx:end_idx, 'percentage'] = interpolated

moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  365 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


In [70]:
moon_phases

,date,percentage
0,2020-01-01,NaN
1,2020-01-02,0.5000
2,2020-01-03,0.5625
3,2020-01-04,0.6250
4,2020-01-05,0.6875
...,...,...
361,2020-12-27,0.8750
362,2020-12-28,0.9375
363,2020-12-29,1.0000
364,2020-12-30,1.0000


In [ ]:
meteor_showers.info()

In [ ]:
moon_phases.info()

In [ ]:
cities.info()

In [ ]:
constellations.info()

In [ ]:
if 'San Diego' in cities.values:
    print('here')
else:
    print('not here')

### 예측 함수
입력된 도시의 Latitude를 구하고  
=> 해당 Latitude에서 관측 가능한 별자리를 구한 후  
=> 각 별자리에 해당하는 유성우를 구해서  
=> 관측 가능 날짜와 달의 위상 등을 조합해서 출력문을 작성한다.

In [74]:
def predict_best_meteor_shower_viewing(city):
    # Create an empty string to return the message back to the user
    meteor_shower_string = ""

    if city not in cities.values:
        meteor_shower_string = "Unfortunately, " + city + " isn't available for a prediction at this time."
        return meteor_shower_string

    # Get the latitude of the city from the cities dataframe
    latitude = cities.loc[cities['city'] == city, 'latitude'].iloc[0]

    # Get the list of constellations that are viewable from that latitude
    constellation_list = constellations.loc[(constellations['latitudestart'] >= latitude) & (constellations['latitudeend'] <= latitude), 'constellation'].tolist()

    # If no constrllations are viewable, let the user know
    if not constellation_list:
        meteor_shower_string = "Unfortunately, there are no meteor showers viewable from "+ city + "."

        return meteor_shower_string

    meteor_shower_string = "In " + city + " you can see the following meteor showers:\n"
    
    # Iterate through each constellation that is viewable from the city
    for constellation in constellation_list:
        # Find ALL meteor showers that are nearest that constellation
        meteor_showers_for_constellation = meteor_showers.loc[meteor_showers['radiant'] == constellation]
        
        # If no meteor showers found for this constellation, skip
        if meteor_showers_for_constellation.empty:
            continue
        
        # Iterate through each meteor shower for this constellation
        for idx, row in meteor_showers_for_constellation.iterrows():
            meteor_shower = row['name']
            meteor_shower_startdate = row['startdate']
            meteor_shower_enddate = row['enddate']

            # Find the moon phases for each date within the viewable timeframe of that meteor shower
            moon_phases_list = moon_phases.loc[(moon_phases['date'] >= meteor_shower_startdate) & (moon_phases['date'] <= meteor_shower_enddate)]
            
            if moon_phases_list.empty:
                continue

            if meteor_shower == 'Chang\'e':
                # For the Chang'e meteor shower, find the date where the moon is the most visible
                best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmax()]['date']
                meteor_shower_string += "Though the moon will be bright, the " + meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
            else:
                # Find the date where the moon is the least visible
                best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmin()]['date']
                meteor_shower_string += meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
    
    return meteor_shower_string

In [75]:
print(predict_best_meteor_shower_viewing('Tokyo'))

In Tokyo you can see the following meteor showers:
Lyrids is best seen if you look towards the Lyra constellation on April 22, 2020.
Eta Aquarids is best seen if you look towards the Aquarius constellation on April 22, 2020.
Orionids is best seen if you look towards the Orion constellation on October 16, 2020.
Perseids is best seen if you look towards the Perseus constellation on July 20, 2020.
Though the moon will be bright, the Chang'e is best seen if you look towards the Draco constellation on October 01, 2020.
Though the moon will be bright, the Chang'e is best seen if you look towards the Draco constellation on October 01, 2020.
Though the moon will be bright, the Chang'e is best seen if you look towards the Draco constellation on October 01, 2020.
Though the moon will be bright, the Chang'e is best seen if you look towards the Draco constellation on October 01, 2020.

